Imports

In [59]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from yeast8model import Yeast8Model

# Initialise model

In [60]:
glc_exch_rate = 16.89
wt_ec = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

Growth (r_2111) unrestricted.


# Ablate and store fluxes in each round

In [61]:
wt_ec.ablation_result = wt_ec.ablate()

ablation_fluxes = wt_ec.ablation_fluxes
ablation_fluxes_diff = ablation_fluxes.copy()
ablation_fluxes_diff.pop("original")
for biomass_component, fluxes in ablation_fluxes_diff.items():
    ablation_fluxes_diff[biomass_component] = (
        ablation_fluxes[biomass_component] - ablation_fluxes["original"]
    )
    print(f"{biomass_component}")
    print(f"min {1e5 * ablation_fluxes_diff[biomass_component].min()} * 1e-5")
    print(f"max {1e5 * ablation_fluxes_diff[biomass_component].max()} * 1e-5")

lipid
min -15.7132268693454 * 1e-5
max 10.732222184671604 * 1e-5
protein
min -15.7132268693454 * 1e-5
max 20.912808268113988 * 1e-5
carbohydrate
min -15.7132268693454 * 1e-5
max 69.90862690764894 * 1e-5
DNA
min -15.7132268693454 * 1e-5
max 87.830895004639 * 1e-5
RNA
min -15.7132268693454 * 1e-5
max 63.041277234143564 * 1e-5
cofactor
min -8.830759742568564 * 1e-5
max 60.46997765979392 * 1e-5
ion
min -15.7132268693454 * 1e-5
max 89.96004436577562 * 1e-5


Work

In [62]:
from script_abl_vs_enz_use import (
    subsystem_dict,
    extract_protein_ids,
    get_participating_rxn_ids,
    get_subsystem_list,
)

In [63]:
list_participating_rxn_df = []

for idx, (biomass_component, fluxes) in enumerate(ablation_fluxes_diff.items()):
    s = fluxes.copy()
    
    enz_metabolite_ids = extract_protein_ids(s)
    participating_rxn_ids, enz_usage_fluxes = get_participating_rxn_ids(
        enz_metabolite_ids, s, wt_ec
    )
    subsystem_list = get_subsystem_list(participating_rxn_ids, subsystem_dict)
    
    enz_usage_flux_column = "enz_usage_flux_" + biomass_component
    participating_rxn_df = pd.DataFrame(
        {
            "participating_rxn_id": participating_rxn_ids,
            "subsystem": subsystem_list,
            enz_usage_flux_column: enz_usage_fluxes,
        }
    )
    list_participating_rxn_df.append(participating_rxn_df)
    
left_columns = list_participating_rxn_df[0].iloc[:,0:2]
enz_usage_columns = pd.concat(
    [list_participating_rxn_df[idx].iloc[:,-1]
     for idx in range(len(list_participating_rxn_df))
    ], axis=1
)

all_fluxes_df = pd.concat([left_columns, enz_usage_columns], axis=1)
all_fluxes_df = all_fluxes_df[all_fluxes_df.subsystem != "Enzyme usage"]
all_fluxes_df = all_fluxes_df.sort_values(by=['subsystem'])

In [ ]:
# 2d proof of concept
fig, ax = plt.subplots(figsize=(10,50))
sns.heatmap(all_fluxes_df.iloc[:,2:], center=0, robust=True, cmap='RdBu')

In [70]:
subsystem_list_sorted = all_fluxes_df.subsystem.to_list()

In [71]:
# TODO: use this without crashing
subsystem_labels = []
compare_string = ''
for subsystem_string in subsystem_list_sorted:
    if subsystem_string == compare_string:
        subsystem_string = ' '
    else:
        compare_string = subsystem_string
    subsystem_labels.append(subsystem_string)